Trace File Cleaning Program

In [116]:
import pandas as pd

# Read the Excel file
file_path = "URS_Export.xlsx"
sheet_name = "Document Preflight"  # Change to the desired sheet name
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Print the first few rows of the DataFrame
print(df.head())

           ID Class       Type  \
0  UTICA-2301   NaN    Heading   
1  UTICA-2302   NaN    Heading   
2  UTICA-2303   NaN    Heading   
3  UTICA-2304   NaN  Info Item   
4  UTICA-2305   NaN    Heading   

                                         Description  Starkey Priority  \
0                                                NaN               NaN   
1                                                NaN               NaN   
2                                                NaN               NaN   
3  Info: The purpose of this document is to defin...               NaN   
4                                                NaN               NaN   

   Verification Allocation                                  Linked Work Items  \
0                      NaN  is branched from: TULUM-1, has branch: XANADU-...   
1                      NaN  is branched from: TULUM-2, has parent: UTICA-2...   
2                      NaN  is branched from: TULUM-3, has parent: UTICA-2...   
3                      NaN

In [117]:
keeper=["ID","Linked Work Items"]
dfc=(
    df
    .loc[df.Type=="User Requirement"]
    .loc[df.Status=="Approved"]
    .loc[:,["ID","Linked Work Items"]]
    .set_index("ID")
    .replace({' ':" "},regex=True)
    .apply(lambda x: x.str.split(',').explode())
    .reset_index()
)
dfc1 = dfc['Linked Work Items'].str.split(':',n=1,expand=True)
dfc1.columns=['Relationship','Link']
dfc2 = dfc.join(dfc1)
dfc2=dfc2.drop(columns=['Linked Work Items'])
dfc2.Relationship=dfc2.Relationship.str.strip()

In [118]:
cat_list = ['is refined by','is validated by']
dfc3 = dfc2[dfc2['Relationship'].isin(cat_list)]



In [119]:
dfc4 = dfc3[dfc3['Link'].str.contains('MASTER|UTICA', case=False)].reset_index(drop=True)
dfc4 = dfc4.rename(columns={'ID': 'UID'})

In [120]:
display(dfc4.head(10))

,UID,Relationship,Link
0,UTICA-6118,is refined by,MASTER-17121
1,UTICA-6117,is refined by,MASTER-17121
2,MASTER-6167,is refined by,MASTER-16473
3,MASTER-6167,is refined by,MASTER-20049
4,MASTER-6167,is refined by,MASTER-20059
5,MASTER-6167,is refined by,MASTER-20061
6,MASTER-6167,is validated by,MASTER-56201
7,MASTER-6168,is refined by,MASTER-20049
8,MASTER-6168,is validated by,MASTER-56201
9,UTICA-6378,is refined by,UTICA-2495


In [121]:
dfc4.to_csv("URS_Cleaned.csv",index=False)
